**Vamos a hacer que cada línea del dataset sea un párrafo de la web scrapeada. Vamos a obtener una nueva lista de palabras confusas y entrenar el modelo luego de haberlas excluido**

In [1]:
clases = 5

In [2]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [4]:
 def limpiarFecha(st):    
        import re
        p1 = r"((\d+)\s(bc|ac)\s)"
        p2 = r"((\d+)\s(bc|ac))$"
        patron = re.compile("|".join([p1,p2]),re.IGNORECASE)
        for i in range(len(st)):
            #print("ciclo",i)
            #print(st)
            if re.search(p1 , st[i:] , re.IGNORECASE):
                var = patron.search(st[i:] , re.IGNORECASE)
                # print(var,len(var[1].strip()),var[1],"|",var[2],"|",var[3])
                new = "".join([var[2],var[3]])
                st  = st.replace(var[1].strip(),new,1)
                print(st)
        return st

In [114]:
def limpiarEdad(st):
    #if type(st) == str: 
    #    st = st.split()
    #if type(st) not in {list,tuple}: return st
    res = []
    
    for t in range(500):
        st = st.replace("[" + str(t) + "]", " ")
    for letra in ["a","b","d","e","f","g","h","i","j"]:
        st = st.replace("[" + letra + "]", " ")
    caracteresARemover = ["(",")","[","]",".",",",'"',"“"]
    for signo in caracteresARemover:
        st = st.replace(signo," ")
    st = st.split()
    for i in range(len(st) - 1):
        if st[i].isdecimal() and ( st[i+1].lower() == "ad" or 
                                   st[i+1].lower() == "bc" or 
                                   st[i+1].lower() == "ac" or 
                                   st[i+1].lower() == "dc"):
            res.append("".join([st[i],st[i+1]]))
        elif st[i] not in res and st[i].lower() not in {"ad","bc","ac","dc"}:
            res.append(st[i])
    if st[-1] not in res  and st[-1].lower() not in {"ad","bc","ac","dc"}: 
        res.append(st[-1])
    return " ".join(res)

In [81]:
limpiarEdad("The Roman–Parthian Wars (54BC – 217 AD) were a series of conflicts  [1]   200 ad. 300 bc)")

'The Roman–Parthian Wars 54BC – 217AD were a series of conflicts 200ad 300bc'

In [54]:
limpiarEdad("a in 42 BC. campaign 42 bc) against Parthia, but his assassination averted the war. During the ensuing Liberators' civil war, the Parthians actively supported Brutus and Cassius, sending a contingent which fought with them at the Battle of Philippi in 42 BC.[11]\n")

"a in 42BC. BC. campaign 42bc) bc) against Parthia, but his assassination averted the war. During ensuing Liberators' civil war, Parthians actively supported Brutus and Cassius, sending contingent which fought with them at Battle of Philippi 42 BC.[11]"

In [51]:
class Textos:
    
    def __init__(self, clases, webs):
        self.clases = clases
        self.lista_dataset = []
        self.crear_dataframe(clases, webs)

    def crear_dataframe(self, clases, webs):
        self.lista_dataset.append('TextoCrudo')
        self.lista_dataset.append('TextoLimpio')

        for i in range(1, clases + 1):
            self.lista_dataset.append('Clase' + str(i))

        self.lista_dataset.append('Predicción')
        self.lista_dataset.append('CLASE')
        self.dataframe = pd.DataFrame(columns=self.lista_dataset)
        self.webs = pd.ExcelFile(webs)
        df = self.webs.parse(self.webs.sheet_names[0])
        self.__clases_df = set(df["CÓDIGO"].value_counts().index)

        for i in df.index:
            print(f"Creando Dataframe {i+1} de {len(df.index)}")
            url = df["URL"][i]
            req = get(url)
            statusCode = req.status_code
            html = BeautifulSoup(req.text, "html.parser")

            if statusCode == 200:
                paragraphs = html.select("p")
                cont = 0

                for para in paragraphs:
                    if len(para.text) < 15:
                        continue
                    cont += 1
                    self.dataframe = self.dataframe.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)

                clear_output(wait=True)

    def limpiar(textoCrudo):
        for i in range(100):
            textoCrudo = textoCrudo.replace("[" + str(i) + "]", "")
        
        for i in ["a","b","c"]:
            textoCrudo = textoCrudo.replace("[" + i + "]", "")
        
        stopwords_english = stopwords.words('english')
        stop        = stopwords.words('english')
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()    
        texto_tokens = tokenizer.tokenize(textoCrudo)
        sufijos_fecha = ["bc","ac"]

        for i in range(len(texto_tokens.copy())):
            if i > len(texto_tokens) - 1:
                break
            if i !=0:
                if texto_tokens[i] in sufijos_fecha and texto_tokens[i - 1].isnumeric():
                    texto_tokens[i] = texto_tokens[i - 1] + texto_tokens[i]     
                    texto_tokens.pop(i - 1)
                           
        texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
        texto_stem   = [stemmer.stem(word) for word in texto_clean] 
        return texto_stem       
      

In [9]:
lista_dataset = []
lista_dataset.append('TextoCrudo')
lista_dataset.append('TextoLimpio')
for i in range(1, clases + 1):
    lista_dataset.append('Clase' + str(i))
lista_dataset.append('Predicción')
lista_dataset.append('CLASE')
dataFrame = pd.DataFrame(columns = lista_dataset)

In [10]:
textos = dataFrame

In [83]:
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars (54 – 217AD) AD) were ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and Roman ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"In 113AD, AD, the Roman Emperor Trajan made ea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,"In 195, another Roman invasion of Mesopotamia ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,"In February 2019, scientists reported evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [ ]:
webs = pd.ExcelFile("Webs.xls")

In [ ]:
df = webs.parse(webs.sheet_names[0])
clases_df = set(df["CÓDIGO"].value_counts().index)
clases_df

In [ ]:
df = df.iloc[12:60,:]

In [ ]:
df = df.drop([55,56,57])
df

In [37]:
df.to_csv("miniWEBs.csv")

In [38]:
df = pd.read_csv("miniWEBs.csv")
df = df.iloc[:,1:]
df

,URL,NOMBRE ARCHIVO,CLASE,CÓDIGO
0,https://en.wikipedia.org/wiki/Roman%E2%80%93Pa...,Roman%E2%80%93Parthian_Wars,EDAD ANTIGUA,1
1,https://en.wikipedia.org/wiki/Celts,Celts,EDAD ANTIGUA,1
2,https://en.wikipedia.org/wiki/Early_Middle_Ages,Early_Middle_Ages,EDAD MEDIA,2
3,https://en.wikipedia.org/wiki/Late_Middle_Ages,/Late_Middle_Ages,EDAD MEDIA,2
4,https://en.wikipedia.org/wiki/Philip_II_of_Spain,Philip_II_of_Spain,EDAD MODERNA,3
5,https://en.wikipedia.org/wiki/Louis_XIV,Louis_XIV,EDAD MODERNA,3
6,https://en.wikipedia.org/wiki/Spanish_American...,Spanish_American_wars_of_independence,EDAD CONTEMPORÁNEA,4
7,https://en.wikipedia.org/wiki/World_War_I,World_War_I,EDAD CONTEMPORÁNEA,4
8,https://en.wikipedia.org/wiki/Prehistory,Prehistory,PREHISTORIA,5
9,https://en.wikipedia.org/wiki/Paleolithic,Paleolithic,PREHISTORIA,5


In [13]:
for i in df.index:
    print(f"Creando Dataframe {i+1} de {len(df.index)}")
    url = df["URL"][i]
    req = get(url)
    statusCode = req.status_code
    if statusCode == 200:
        html = BeautifulSoup(req.text, "html.parser")
        paragraphs = html.select("p")
        cont = 0
        for para in paragraphs:
            if len(para.text) < 40:
                continue
            cont += 1
            textos = textos.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)
    clear_output(wait=True)
textos 

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars (54 BC – 217 AD) were ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"In 113 AD, the Roman Emperor Trajan made easte...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,"In 195, another Roman invasion of Mesopotamia ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,"In February 2019, scientists reported evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [14]:
textos.sample(10) 

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
458,"By the time of the Treaty of Ryswick, the Span...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
876,The Paleolithic is often held to finish at the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
810,"In 1915 John McCrae, a Canadian army doctor, w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
904,Upper Paleolithic humans produced works of art...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
662,Russian armies generally had success in the Ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
681,The Czechoslovak Legion fought on the side of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
803,"Like all the armies of mainland Europe, Austri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
905,R. Dale Guthrie[77] has studied not only the m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
431,France also attempted to participate actively ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
653,"Faced with Russia in the east, Austria-Hungary...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [154]:
textos.to_csv("MiniHistoriaTextosCrudos.csv")

In [116]:
textos = pd.read_csv("MiniHistoriaTextosCrudos.csv")
textos = textos.iloc[:,1:]
textos.head(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars (54 BC – 217 AD) were ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"In 113 AD, the Roman Emperor Trajan made easte...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,"In 195, another Roman invasion of Mesopotamia ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,By the same time the Parthians started their r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,"After 90 BC, the Parthian power was diminished...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,When Pompey took charge of the war in the East...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,"In 53 BC, Crassus led an invasion of Mesopotam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,"The following year, the Parthians launched rai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [40]:
textos["TextoCrudo"][10]

"During Caesar's civil war the Parthians made no move, but maintained relations with Pompey. After his defeat and death, a force under Pacorus came to the aid of the Pompeian general Caecilius Bassus, who was besieged at Apamea Valley by the Caesarian forces. With the civil war over, Julius Caesar elaborated plans for a campaign against Parthia, but his assassination averted the war. During the ensuing Liberators' civil war, the Parthians actively supported Brutus and Cassius, sending a contingent which fought with them at the Battle of Philippi in 42 BC.[11]\n"

In [111]:
for i in textos.index:
    clear_output(wait=True)        
    print(f"Limpiando Texto Crudo {i+1} de {len(textos.index)}")
    textos["TextoCrudo"][i] = limpiarEdad(textos["TextoCrudo"][i])
textos

Limpiando Texto Crudo 922 de 922


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and Roman ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [155]:
conta = 0
for i in range(1, len(textos.index)):
    conta += 1
    if conta > 100000:
        break
    print()
    print()
    print(textos["TextoCrudo"][i])
    print()
    print(limpiarEdad(textos["TextoCrudo"][i]))



Battles between the Parthian Empire and the Roman Republic began in 54 BC.[1] This first incursion against Parthia was repulsed, notably at the Battle of Carrhae (53 BC). During the Roman Liberators' civil war of the 1st Century BC, the Parthians actively supported Brutus and Cassius, invading Syria, and gaining territories in the Levant.  However, the conclusion of the second Roman civil war brought a revival of Roman strength in Western Asia.[2]


Battles between the Parthian Empire and Roman Republic began in 54BC This first incursion against Parthia was repulsed notably at Battle of Carrhae 53BC During Liberators' civil war 1st Century Parthians actively supported Brutus Cassius invading Syria gaining territories Levant However conclusion second brought a revival strength Western Asia


In 113 AD, the Roman Emperor Trajan made eastern conquests and the defeat of Parthia a strategic priority,[3] and successfully overran the Parthian capital, Ctesiphon, installing Parthamaspates of



The interlace patterns that are often regarded as typical of "Celtic art" were characteristic of the whole of the British Isles, a style referred to as Insular art, or Hiberno-Saxon art. This artistic style incorporated elements of La Tène, Late Roman, and, most importantly, animal Style II of Germanic Migration Period art. The style was taken up with great skill and enthusiasm by Celtic artists in metalwork and illuminated manuscripts. Equally, the forms used for the finest Insular art were all adopted from the Roman world: Gospel books like the Book of Kells and Book of Lindisfarne, chalices like the Ardagh Chalice and Derrynaflan Chalice, and penannular brooches like the Tara Brooch. These works are from the period of peak achievement of Insular art, which lasted from the 7th to the 9th centuries, before the Viking attacks sharply set back cultural life.


The interlace patterns that are often regarded as typical of Celtic art were characteristic the whole British Isles a style re

Hunyadi succeeded during the Siege of Belgrade in 1456 against Ottomans biggest victory that empire decades This battle became a real Crusade Muslims as peasants were motivated by Franciscan friar Saint John Capistrano who came from Italy predicating Holy War The effect it created time was one main factors helped achieving However premature death Hungarian Lord left Pannonia defenseless and chaos In an extremely unusual event for Middle Ages Hunyadi's son Matthias elected King Hungary nobility For first member aristocratic family not royal crowned


King Matthias Corvinus of Hungary (1458–1490) was one of the most prominent figures of the period, directing campaigns to the West, conquering Bohemia in answer to the Pope's call for help against the Hussite Protestants. Also, in resolving political hostilities with the German emperor Frederick III of Habsburg, he invaded his western domains. Matthew organized the Black Army of mercenary soldiers; it was considered as the biggest army of i

Rapid French advance led to a coup that toppled De Witt and brought William III power Leopold viewed expansion into the Rhineland as an increasing threat especially after their seizure of strategic Duchy Lorraine in 1670 The prospect Dutch defeat alliance with Brandenburg-Prussia on 23 June followed by another Republic 25th Although Brandenburg was forced out war 1673 Treaty Vossem August anti-French formed Spain Emperor Duke


The French alliance was deeply unpopular in England, who made peace with the Dutch in the February 1674 Treaty of Westminster. However, French armies held significant advantages over their opponents; an undivided command, talented generals like Turenne, Condé and Luxembourg and vastly superior logistics. Reforms introduced by Louvois, the Secretary of War, helped maintain large field armies that could be mobilised much quicker, allowing them to mount offensives in early spring before their opponents were ready.[37]


The French alliance was deeply unpopular in E

Louis is a member of the House Bourbon branch Capetian dynasty and Robertians


Louis' patriline is the line from which he is descended father to son. It follows the Bourbon, Kings of France, and the Counts of Paris and Worms. This line can be traced back more than 1,200 years from Robert of Hesbaye to the present day, through Kings of France & Navarre, Spain and Two-Sicilies, Dukes of Parma and Grand-Dukes of Luxembourg, Princes of Orléans and Emperors of Brazil. It is one of the oldest in Europe.


Louis' patriline is the line from which he descended father to son It follows Bourbon Kings of France and Counts Paris Worms This can be traced back more than 1 200 years Robert Hesbaye present day through & Navarre Spain Two-Sicilies Dukes Parma Grand-Dukes Luxembourg Princes Orléans Emperors Brazil one oldest in Europe


This is an incomplete list of Louis XIV's illegitimate children. He reputedly had more, but the difficulty in fully documenting all such births restricts the list only t


In late 1915 a Franco-British force landed at Salonica in Greece to offer assistance and pressure its government declare war against the Central Powers However pro-German King Constantine I dismissed pro-Allied of Eleftherios Venizelos before Allied expeditionary arrived The friction between Allies continued accumulate with National Schism which effectively divided regions still loyal king new provisional After intense negotiations an armed confrontation Athens royalist forces incident known as Noemvriana resigned his second son Alexander took place; officially joined on side June 1917


The Macedonian front was initially mostly static. French and Serbian forces retook limited areas of Macedonia by recapturing Bitola on 19 November 1916 following the costly Monastir Offensive, which brought stabilisation of the front.[135]


The Macedonian front was initially mostly static French and Serbian forces retook limited areas of Macedonia by recapturing Bitola on 19 November 1916 following t

that war should be avoided at almost any cost would solve nothing the whole of Europe and more besides reduced to ruin loss life so large populations decimated In our ignorance I many us felt ashamed a British General who uttered such depressing unpatriotic sentiments but during next four years those survived holocaust—probably not than one-quarter us—learned how right General's prognosis was courageous he had been utter it


Voicing these sentiments did not hinder Smith-Dorrien's career, or prevent him from doing his duty in World War I to the best of his abilities.


Voicing these sentiments did not hinder Smith-Dorrien's career or prevent him from doing his duty in World War I to the best of abilities


Many countries jailed those who spoke out against the conflict. These included Eugene Debs in the United States and Bertrand Russell in Britain. In the US, the Espionage Act of 1917 and Sedition Act of 1918 made it a federal crime to oppose military recruitment or make any statements

Paleolithic peoples suffered less famine and malnutrition than the Neolithic farming tribes that followed them This was partly because hunter-gatherers accessed a wider variety of natural foods which allowed more nutritious diet decreased risk Many famines experienced by some modern farmers were caused or amplified their dependence on small number crops It is thought wild can have significantly different nutritional profile cultivated The greater amount meat obtained hunting big game animals in diets may also to enjoy agriculturalists has been argued shift from gathering agriculture resulted an increasing focus limited with likely taking back seat plants unlikely affected diseases affluence such as type 2 diabetes coronary heart disease cerebrovascular they ate mostly lean meats frequently engaged intense physical activity average lifespan shorter age common onset these conditions


Large-seeded legumes were part of the human diet long before the Neolithic Revolution, as evident from a

In [ ]:
        stopwords_english = stopwords.words('english')
        stop        = stopwords.words('english')
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()    
        texto_tokens = tokenizer.tokenize(textoCrudo)
        sufijos_fecha = ["bc","ac"]


                           
        texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
        texto_stem   = [stemmer.stem(word) for word in texto_clean] 
        return texto_stem       

In [159]:
st = 'Anthropologists such as Tim White suggest that cannibalism was common in human societies prior to the beginning of the Upper Paleolithic, based on the large amount of “butchered human" bones found in Neanderthal and other Lower/Middle Paleolithic sites.[132] Cannibalism in the Lower and Middle Paleolithic may have occurred because of food shortages.[133] However, it may have been for religious reasons, and would coincide with the development of religious practices thought to have occurred during the Upper Paleolithic.[90][134] Nonetheless, it remains possible that Paleolithic societies never practiced cannibalism, and that the damage to recovered human bones was either the result of excarnation or predation by carnivores such as saber-toothed cats, lions, and hyenas.[90]'

In [145]:
st = 'Anthropologists such as Tim White suggest that cannibalism was common in human societies prior to the beginning of the Upper Paleolithic, based on the large amount of “butchered human" bones found in Neanderthal and other Lower/Middle Paleolithic sites.[132] Cannibalism in the Lower and Middle Paleolithic may have occurred because of food shortages.[133] However, it may have been for religious reasons, and would coincide with the development of religious practices thought to have occurred during the Upper Paleolithic.[90][134] Nonetheless, it remains possible that Paleolithic societies never practiced cannibalism, and that the damage to recovered human bones was either the result of excarnation or predation by carnivores such as saber-toothed cats, lions, and hyenas.[90]'

In [131]:
st = 'Anthropologists such as Tim White suggest that cannibalism was common in human societies'

In [170]:
# st = 'Anthropologists ,such , as Tim [1] White suggest “““that “ cannibalism was common in human societies'

st = "Anthropologists such as Tim White suggest that cannibalism was common in human societies prior to the beginning of the Upper Paleolithic"

print(st)

for t in range(500):
    st = st.replace("[" + str(t) + "]", " ")
for letra in ["a","b","d","e","f","g","h","i","j"]:
    st = st.replace("[" + letra + "]", " ")
caracteresARemover = ["(",")","[","]",".",",",'"',"“"]
for signo in caracteresARemover:
    st = st.replace(signo," ")


res = []
st = st.split()
print()
print(st)
for i in range(len(st)):
    if st[i].isdecimal() and ( st[i+1].lower() == "ad" or 
        st[i+1].lower() == "bc" or 
        st[i+1].lower() == "ac" or 
        st[i+1].lower() == "dc"):
        res.append("".join([st[i],st[i+1]]))
        print("sale por el if")
    elif st[i] not in res and st[i].lower() not in {"ad","bc","ac","dc"}:
        print(i,res)  
        res.append(st[i])
        print(i,res)  
    else:    
        print()
        print(i, "sale por el else")
        print(st[i], st[i+1])
        print()

        
if st[-1] not in res  and st[-1].lower() not in {"ad","bc","ac","dc"}: 
    res.append(st[-1])
print()    
print(res)       
print()
print(" ".join(res))

Anthropologists such as Tim White suggest that cannibalism was common in human societies prior to the beginning of the Upper Paleolithic

['Anthropologists', 'such', 'as', 'Tim', 'White', 'suggest', 'that', 'cannibalism', 'was', 'common', 'in', 'human', 'societies', 'prior', 'to', 'the', 'beginning', 'of', 'the', 'Upper', 'Paleolithic']
0 []
0 ['Anthropologists']
1 ['Anthropologists']
1 ['Anthropologists', 'such']
2 ['Anthropologists', 'such']
2 ['Anthropologists', 'such', 'as']
3 ['Anthropologists', 'such', 'as']
3 ['Anthropologists', 'such', 'as', 'Tim']
4 ['Anthropologists', 'such', 'as', 'Tim']
4 ['Anthropologists', 'such', 'as', 'Tim', 'White']
5 ['Anthropologists', 'such', 'as', 'Tim', 'White']
5 ['Anthropologists', 'such', 'as', 'Tim', 'White', 'suggest']
6 ['Anthropologists', 'such', 'as', 'Tim', 'White', 'suggest']
6 ['Anthropologists', 'such', 'as', 'Tim', 'White', 'suggest', 'that']
7 ['Anthropologists', 'such', 'as', 'Tim', 'White', 'suggest', 'that']
7 ['Anthropologists', 